In [1]:
from datasets import load_dataset, Dataset
from tqdm import tqdm
import openai
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

/home/tkdrnjs0621/miniconda3/envs/qwen2a2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [ ]:
dataset = Dataset.from_json('/home/tkdrnjs0621/work/lcva/output_wer.jsonl')
hicupid = load_dataset("12kimih/HiCUPID","dialogue")

Generating train split: 125 examples [00:00, 20962.30 examples/s]


In [ ]:
formatted = {}

for item in tqdm(hicupid['train']):
    user_id = item['user_id']
    dialogue_id = item['dialogue_id']
    
    # Initialize nested structure if not exists
    if user_id not in formatted:
        formatted[user_id] = {}
    if dialogue_id not in formatted[user_id]:
        formatted[user_id][dialogue_id] = []
    
    # Append user and assistant turns as a list
    formatted[user_id][dialogue_id].append(item['user'])
    formatted[user_id][dialogue_id].append(item['assistant'])

formatted_str = {}
for uid, dialogs in formatted.items():
    formatted_str[uid] = {}
    for did, turns in dialogs.items():
        lines = []
        for i, text in enumerate(turns):
            prefix = 'User:' if i % 2 == 0 else 'Assistant:'
            lines.append(f'{prefix} {text}')
        formatted_str[uid][did] = "\n".join(lines)

In [4]:
dataset[0]

{'user_id': '0',
 'dialogue_id': '0',
 'question': 'What did you say about wearing rainboots in snowy conditions?',
 'answer': "Rainboots can be used in snowy conditions, but they may lack insulation. It's recommended to wear thick, thermal socks for added warmth, or consider insulated rainboots for colder climates.",
 'wer': 0.2,
 'asr_result': ' What did you say about wearing rain boots in snowy conditions?',
 'txt_i': ' what did you say about wearing rain boots in snowy conditions',
 'txt_r': 'what did you say about wearing rainboots in snowy conditions'}

In [ ]:
row = dataset[0]
base_path = '/home/tkdrnjs0621/work/lcva/outputs/dialogue_gt/v1/'

In [ ]:
question = row['question']
answer = row['answer']
uid = row['user_id']
did = row['dialogue_id']

file_path = base_path + f"/{uid}_{did}.txt"
try:
    with open(file_path, "r", encoding="utf-8") as file:
        last_line = file.readlines()[-1].strip()
        response=last_line
except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4.1",
    messages= [
    {"role":"system","content":"You are a helpful assistant."},
    {"role": "user", "content": f"""You need to evaluate the performance of a voice assistant in a multi-turn speech interaction scenario. The model receives a speech input from the
user, who is asking about the past utterance and generate response.
Your task is to assess the model's response in the final turn, based on the user's question and the reference answer.
The primary evaluation criterion is how well the model's response includes the key information from the reference answer that is relevant to the user's
question. While the model may provide additional information, it must accurately reflect the essential content from the reference answer that pertains to the user's query.

### Scoring Criteria (1 to 5 scale):
- **1 point**: Fails to include relevant details from the reference answer.
- **2 points**: Includes some relevant details but omits key information from the reference answer.
- **3 points**: Partially includes relevant details but with omissions or misrepresentations of key points.
- **4 points**: Mostly includes the key details from the reference answer, with only minor inaccuracies or omissions.
- **5 points**: Fully includes the key information from the reference answer without any omissions or errors.
Below are the transcriptions of the user's input, the model's response, and the reference answer:
[User Input] {question}
[Response] {response}
[Reference Answer] {answer}
Please output only a single score (1-5) for the conversation without any explanations.
""".strip()},
],
    temperature=1,
    top_p=1,
)

print(completion.choices[0].message.content)

SyntaxError: incomplete input (2957690427.py, line 20)